In [1]:
from keras.models import model_from_yaml
yaml_file = open('model.yaml', 'r')
loaded_model_yaml = yaml_file.read()
yaml_file.close()
model = model_from_yaml(loaded_model_yaml)
# load weights into new model
model.load_weights("model.h5")
print("Loaded model from disk")

Using TensorFlow backend.


Loaded model from disk


In [2]:
def detectSigns(frame,printFlag):
        #noramalization
    frame2 = cv2.medianBlur(frame,3)      
    frame2 = cv2.GaussianBlur(frame2,(3,3),2) #important for finding circles!
        #masking
    hsv_image_1 = cv2.cvtColor(frame2, cv2.COLOR_BGR2HSV)
    low_blue = (95,100,100)
    hight_blue = (120,255,255)
    mask = cv2.inRange(hsv_image_1, low_blue, hight_blue)  
    if(printFlag):
        cv2.imshow('Mask_Signs',mask)  
    circles = cv2.HoughCircles(mask,cv2.HOUGH_GRADIENT,1,50,param1=70,param2=10,minRadius=10,maxRadius=40) #let's change params
    pred=[]
    if np.all(circles) != None:
        circles = np.round(circles[0, :]).astype("int")
        for (x, y, r) in circles:
            if((x-r) > 0 and (x+r) >0 and (y-r)>0 and (y+r)>0):
                cut_frame = frame2[y-r:y+r,x-r:x+r]
                cut_frame = cv2.resize(cut_frame,(32, 32), interpolation = cv2.INTER_CUBIC)
                k = model.predict(np.expand_dims(np.array(cut_frame.astype("float") / 255.0), axis=0))
#                print(k)
                maximum = max(k[0])
                
                index = np.where(k[0] == maximum)[0][0]
                print(str(maximum)+" "+str(index))
                if(maximum>0.9):
                    print(str(maximum)+" "+str(index))
                    pred.append(index)
                    if(printFlag):
                        frame = cv2.rectangle(frame,(x-r,y-r),(x+r,y+r),(0,0,255),2)            
                        cv2.imshow('Sign_cut:'+str(index),cut_frame)
    return pred

In [3]:
import cv2
import numpy as np

cap=cv2.VideoCapture("signs_test.mp4")
frame=cap.read()[1]

while(cap.isOpened() and len(frame)>0):
    frame = cv2.resize(frame,(640,480), interpolation = cv2.INTER_CUBIC)

    pred = detectSigns(frame,True)
    cv2.imshow('frame2',frame)
    if cv2.waitKey(100) & 0xFF == ord("q"):
        break
    frame = cap.read()[1]
    
    
cap.release()        
cv2.destroyAllWindows()